# Elasticsearch Quick Start

<a target="_blank" href="https://colab.research.google.com/github/joemcelroy/elasticsearch-labs/blob/notebooks-guides/colab-notebooks-examples/search/00-quick-start.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This interactive notebook will introduce you to the very basics of getting started with simple Elasticsearch queries, using the official [Elasticsearch Python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html).
We'll run through getting the client up and running, indexing a small data set into Elasticsearch, and performing basic searches against your data.

## Create Elastic Cloud deployment

If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?fromURI=%2Fhome) for a free trial.

- Go to the [Create deployment](https://cloud.elastic.co/deployments/create) page
   - Select **Create deployment**

## Install packages and import modules

To get started, we'll need to connect to our Elastic deployment using the Python client.
Because we're using an Elastic Cloud deployment, we'll use the **Cloud ID** to identify our deployment.

First we need to install the `elasticsearch` Python client.

In [ ]:
!pip install -qU elasticsearch sentence-transformers==2.2.2

# Setup the Embedding Model
For this example, we're using the all-MiniLM-L6-v2, part of the sentence_transformers library. You can read more about this model in [hugging face](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2).

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model

## Initialize the Elasticsearch client

Now we can instantiate the [Elasticsearch python client](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html), providing the cloud id and password in your deployment.

In [30]:
from elasticsearch import Elasticsearch
from getpass import getpass

CLOUD_ID = getpass("Elastic Cloud ID")
CLOUD_PASSWORD = getpass("Elastic Password")

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", CLOUD_PASSWORD)
)

Elastic Cloud ID··········
Elastic Password··········


If you're running Elasticsearch locally or self-managed, you can pass in the Elasticsearch host instead. [Read more](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later) on how to connect to Elasticsearch locally

Confirm that the client has connected with this test.

In [31]:
print(client.info())

{'name': 'instance-0000000000', 'cluster_name': '1a56ad21587c44d3930932eb9fa1d8e8', 'cluster_uuid': 'gX4zlwtlR4qhZpp1SPm4Yg', 'version': {'number': '8.8.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98e1271edf932a480e4262a471281f1ee295ce6b', 'build_date': '2023-06-26T05:16:16.196344851Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## Index some test data

Our client is set up and connected to our Elastic deployment.
Now we need some data to test out the basics of Elasticsearch queries.
We'll use a small index of books with the following fields:

- `title`
- `authors`
- `publish_date`
- `num_reviews`
- `publisher`

### Create index

Let's create an Elasticsearch index with the correct mappings for our test data.

In [ ]:
# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text"},
            "authors": {"type": "keyword"},
            "summary": {"type": "text"},
            "publish_date": {"type": "date"},
            "num_reviews": {"type": "integer"},
            "publisher": {"type": "keyword"},
            "title_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": "true",
                "similarity": "cosine"
            }
        }
    }
}

# Create the index
client.indices.create(index='book_index', body=mapping)


### Index test data

Run the following command to upload some test data, containing information about 10 popular programming books from this [dataset](https://raw.githubusercontent.com/joemcelroy/elasticsearch-labs/notebooks-guides/colab-notebooks-examples/search/data.json).

In [ ]:
import json
from urllib.request import urlopen

url = "https://raw.githubusercontent.com/joemcelroy/elasticsearch-labs/notebooks-guides/colab-notebooks-examples/search/data.json"
response = urlopen(url)
books = json.loads(response.read())

actions = []
for book in books:
    actions.append({"index": {"_index": "book_index"}})
    # Transforming the title into an embedding using the model
    book["title_vector"] = model.encode(book["title"]).tolist()
    actions.append(book)
client.bulk(index="book_index", operations=actions)


## Aside: Pretty printing Elasticsearch responses

Your API calls will return hard-to-read nested JSON.
We'll create a little function called `pretty_response` to return nice, human-readable outputs from our examples.

In [40]:
def pretty_response(response):
    for hit in response['hits']['hits']:
        id = hit['_id']
        publication_date = hit['_source']['publish_date']
        score = hit['_score']
        title = hit['_source']['title']
        summary = hit['_source']['summary']
        publisher = hit["_source"]["publisher"]
        num_reviews = hit["_source"]["num_reviews"]
        authors = hit["_source"]["authors"]
        pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}")
        print(pretty_output)

##Making Queries

Now that we have indexed the books, we want to perform a semantic search for books that similarly match the query. We embed the query and perform a search.

In [41]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100
    }
})

pretty_response(response)


ID: OOlWP4kB-GB5Evg6zHVx
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 0.8075247

ID: NOlWP4kB-GB5Evg6zHVx
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 0.6946182

ID: NelWP4kB-GB5Evg6zHVx
Publication date: 2018-12-04
Title: Eloquent JavaScript
Summary: A modern introduction to programming
Publisher: no starch press
Reviews: 38
Authors: ['marijn haverbeke']
Score: 0.66179085

ID: MOlWP4kB-GB5Evg6zHVx
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486



<ipython-input-41-f6998147c0ac>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


## Filtering

Filter context is mostly used for filtering structured data, for example:

Does this timestamp fall into the range 2015 to 2016?
Is the status field set to "published"?
Filter context is in effect whenever a query clause is passed to a filter parameter, such as the filter or must_not parameters in the bool query.

[Read more](https://)

### Example: Keyword Filtering

This is an example of adding a keyword filter to the query.

It narrows down the results by including only documents where the "publisher" field is equal to "addison-wesley".

Overall, the code retrieves the top books that are similar to "Best javascript books?" based on their title vectors and have "addison-wesley" as the publisher.

In [ ]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100,
      "filter": {
          "term": {
              "publisher": "addison-wesley"
          }
      }
    }
})

pretty_response(response)

### Example: Optional Filtering

TODO: Help the developer understand more about the different types of filtering they can do.

Provide a link to show more advanced use cases of filtering on date-range, geo-location etc.

In [43]:
response = client.search(index="book_index", body={
    "knn": {
      "field": "title_vector",
      "query_vector": model.encode("Best javascript books?"),
      "k": 10,
      "num_candidates": 100,
      "filter": {
          "bool": {
              "should": [
                  {
                    "term": {
                        "publisher": "addison-wesley"
                    }
                  },
                  {
                    "term": {
                        "authors": "robert c. martin"
                    }
                  }
              ],

          }
      }
    }
})

pretty_response(response)


ID: MOlWP4kB-GB5Evg6zHVx
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486

ID: N-lWP4kB-GB5Evg6zHVx
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Publisher: prentice hall
Reviews: 20
Authors: ['robert c. martin']
Score: 0.57042736

ID: NulWP4kB-GB5Evg6zHVx
Publication date: 1994-10-31
Title: Design Patterns: Elements of Reusable Object-Oriented Software
Summary: Guide to design patterns that can be used in any object-oriented language
Publisher: addison-wesley
Reviews: 45
Authors: ['erich gamma', 'richard helm', 'ralph johnson', 'john vlissides']
Score: 0.56175697

ID: M-lWP4kB-GB5Evg6zHVx
Publication date: 2008-08-11
Title: Clean Code: A Handbo

<ipython-input-43-9b650e64895f>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


## Hybrid Search

In this example, we are investigating the combination of two search algorithms: BM25 for text search and HNSW for nearest neighbor search. By combining multiple ranking methods, such as BM25 and an ML model that generates dense vector embeddings, we can achieve the best ranking results. This approach allows us to leverage the strengths of each algorithm and improve the overall search performance.

TODO: Explain why we use RRF here


In [51]:
response = client.search(index="book_index", body={
    "query": {
        "match": {
            "summary": "python"
        }
    },
    "knn": {
        "field": "title_vector",
        # generate embedding for query so it can be compared to `title_vector`
        "query_vector" : model.encode("python programming").tolist(),
        "k": 5,
        "num_candidates": 10
    },
    "rank": {
        "rrf": {
            "window_size": 100,
            "rank_constant": 20
        }
    }
})

pretty_response(response)


ID: MelWP4kB-GB5Evg6zHVx
Publication date: 2019-05-03
Title: Python Crash Course
Summary: A fast-paced, no-nonsense guide to programming in Python
Publisher: no starch press
Reviews: 42
Authors: ['eric matthes']
Score: None

ID: MOlWP4kB-GB5Evg6zHVx
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: None

ID: OelWP4kB-GB5Evg6zHVx
Publication date: 2012-06-27
Title: Introduction to the Theory of Computation
Summary: Introduction to the theory of computation and complexity theory
Publisher: cengage learning
Reviews: 33
Authors: ['michael sipser']
Score: None

ID: N-lWP4kB-GB5Evg6zHVx
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Publisher: prentice hall
Review

<ipython-input-51-3861c4af16a3>:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = client.search(index="book_index", body={


In [52]:
client.indices.delete(index="book_index")

ObjectApiResponse({'acknowledged': True})